**Kết quả**

Mô hình chưa nhận diện với độ chính xác chấp nhận được đối với hình ảnh từ tập dữ liệu Test của Fruits360.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
path = '/content/drive/MyDrive/Colab Notebooks/Fruits360/Lite 15.03.2025'
print(os.listdir(path))

['Test', 'Training']


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = path + '/Training'
validation_dir = path + '/Test'

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(100, 100),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(100, 100),
    batch_size=32,
    class_mode='categorical'
)

Found 5461 images belonging to 11 classes.
Found 1828 images belonging to 11 classes.


In [ ]:
model_cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(11, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model_cnn.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model_cnn.fit(train_generator, epochs=10, validation_data=validation_generator)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 1845s 11s/step - accuracy: 0.7541 - loss: 0.8453 - val_accuracy: 0.9606 - val_loss: 0.0872
Epoch 2/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 116s 677ms/step - accuracy: 0.9998 - loss: 0.0031 - val_accuracy: 0.9683 - val_loss: 0.0596
Epoch 3/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 114s 669ms/step - accuracy: 1.0000 - loss: 3.7100e-04 - val_accuracy: 0.9683 - val_loss: 0.0674
Epoch 4/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 122s 711ms/step - accuracy: 1.0000 - loss: 1.7097e-04 - val_accuracy: 0.9694 - val_loss: 0.0644
Epoch 5/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 136s 676ms/step - accuracy: 1.0000 - loss: 8.5098e-05 - val_accuracy: 0.9705 - val_loss: 0.0620
Epoch 6/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 142s 678ms/step - accuracy: 1.0000 - loss: 5.8226e-05 - val_accuracy: 0.9737 - val_loss: 0.0567
Epoch 7/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 114s 666ms/step - accuracy: 1.0000 - loss: 3.5790e-05 - val_accuracy: 0.9721 - val_loss: 0.0580
Epoch 8/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 116s 678ms/step -

In [ ]:
print(train_generator.class_indices)

{'Apple 10': 0, 'Apple 13': 1, 'Apple 14': 2, 'Apple 6': 3, 'Banana 1': 4, 'Banana 3': 5, 'Banana Lady Finger 1': 6, 'Banana Red 1': 7, 'Lemon 1': 8, 'Lemon Meyer 1': 9, 'Orange 1': 10}


In [ ]:
import tensorflow as tf
converter_tflite_model = tf.lite.TFLiteConverter.from_keras_model(model_cnn)
tflite_model = converter_tflite_model.convert()

Saved artifact at '/tmp/tmpf0krv2jd'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100, 100, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 11), dtype=tf.float32, name=None)
Captures:
  137012949827664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137012949829776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137012949828624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137012949827856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137012949829200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137012949828816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137012949830160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137012949829392: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [ ]:
model_cnn.save('/content/drive/MyDrive/Colab Notebooks/Results/fruits360_model_2.h5')

In [ ]:
import cv2 as cv
import numpy as np

class_names = [
    'Apple 10', 'Apple 13', 'Apple 14', 'Apple 6',
    'Banana 1', 'Banana 3', 'Banana Lady Finger 1', 'Banana Red 1',
    'Lemon 1', 'Lemon Meyer 1', 'Orange 1'
]

model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Results/fruits360_model_2.h5')
image = cv.imread('/content/drive/MyDrive/Colab Notebooks/Fruits360/Lite 15.03.2025/lemon.jpg')
image = cv.resize(image, (100, 100)) / 255.0
image = np.expand_dims(image, axis=0).astype(np.float32)
predictions = model.predict(image)
print(predictions[0])

class_idx = np.argmax(predictions[0])
class_name = class_names[class_idx]
confidence = predictions[0][class_idx] * 100
print(f"Result: {class_name}: {confidence:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
[1.2696681e-09 9.9008041e-30 8.6973111e-18 2.7904930e-08 8.7097747e-04
 2.6153307e-02 2.7517224e-06 9.7297299e-01 9.5226730e-15 4.9328492e-19
 5.4774311e-22]
Result: Banana Red 1: 97.30%
